In [1]:
import os
import json
import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/ESTSoft/Data/

[Errno 2] No such file or directory: '/content/drive/MyDrive/ESTSoft/Data/'
/content


In [2]:
TRAIN_JSON_PATH = '/content/final_data/label/train/'
VALID_JSON_PATH = '/content/final_data/label/val/'

TRAIN_IMG_PATH = '/content/final_data/img/train/'
VALID_IMG_PATH = '/content/final_data/img/val/'

TRAIN_SEGMENT_PATH = '/content/final_data/segmentation/train/'
VALID_SEGMENT_PATH = '/content/final_data/segmentation/val/'

In [3]:
with open (TRAIN_JSON_PATH + "train_anger.json", "r", encoding='euc-kr') as f:
    anger_data = json.load(f)
with open (TRAIN_JSON_PATH + "train_happy.json", "r", encoding='euc-kr') as f:
    happy_data = json.load(f)
with open (TRAIN_JSON_PATH + "train_panic.json", "r", encoding='euc-kr') as f:
    panic_data = json.load(f)
with open (TRAIN_JSON_PATH + "train_sadness.json", "r", encoding='euc-kr') as f:
    sadness_data = json.load(f)

In [4]:
# JSON 데이터를 DataFrame으로 변환
anger_df = pd.DataFrame(anger_data)
happy_df = pd.DataFrame(happy_data)
panic_df = pd.DataFrame(panic_data)
sadness_df = pd.DataFrame(sadness_data)

In [5]:
# 각 DataFrame을 리스트에 저장
dfs = [anger_df, happy_df, panic_df, sadness_df]

# DataFrame 합치기
train_df = pd.concat(dfs, ignore_index=True)
train_df = train_df.sort_values(by='filename')
train_df.head()

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
3398,007b10b45d424e7876e73b6b66d62d603c6ef2ee886761...,여,20,일반인,당황,숙박 및 거주공간,"{'boxes': {'maxX': 1888.9304000000002, 'maxY':...","{'boxes': {'maxX': 1888.8512363165685, 'maxY':...","{'boxes': {'maxX': 1888.9304, 'maxY': 1413.411..."
643,00ceb5eb09f1ee2f4144cbda7fd5561fb1b0252310231e...,여,30,일반인,분노,공공시설/종교/의료시설,"{'boxes': {'maxX': 1808.2333000000008, 'maxY':...","{'boxes': {'maxX': 1792.0642309314867, 'maxY':...","{'boxes': {'maxX': 1808.2332999999996, 'maxY':..."
1264,00hj57bc2be60739b1b9f99316a3a11de5febe993b0938...,여,40,일반인,분노,행사/사무공간,"{'boxes': {'maxX': 1843.9142463679543, 'maxY':...","{'boxes': {'maxX': 1866.0104193116026, 'maxY':...","{'boxes': {'maxX': 1852.6880000000003, 'maxY':..."
5596,00skd6f03c1bc8e3f618fb80b7863679bf684363a4d013...,여,20,전문인,슬픔,문화재 및 유적지,"{'boxes': {'maxX': 1624.9164, 'maxY': 1418.914...","{'boxes': {'maxX': 1621.3714458715597, 'maxY':...","{'boxes': {'maxX': 1624.9163999999998, 'maxY':..."
4749,00yucd996df0c12e2dcac91e0a41d77a35181e5ff4cce6...,남,40,전문인,슬픔,실외 자연환경,"{'boxes': {'maxX': 2250.07, 'maxY': 1605.0603,...","{'boxes': {'maxX': 2250.2525267323604, 'maxY':...","{'boxes': {'maxX': 2250.1939980518337, 'maxY':..."


In [6]:
anger_df = anger_df.sort_values(by='filename')
anger_df.reset_index(drop=True, inplace=True)
anger_df

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
0,00ceb5eb09f1ee2f4144cbda7fd5561fb1b0252310231e...,여,30,일반인,분노,공공시설/종교/의료시설,"{'boxes': {'maxX': 1808.2333000000008, 'maxY':...","{'boxes': {'maxX': 1792.0642309314867, 'maxY':...","{'boxes': {'maxX': 1808.2332999999996, 'maxY':..."
1,00hj57bc2be60739b1b9f99316a3a11de5febe993b0938...,여,40,일반인,분노,행사/사무공간,"{'boxes': {'maxX': 1843.9142463679543, 'maxY':...","{'boxes': {'maxX': 1866.0104193116026, 'maxY':...","{'boxes': {'maxX': 1852.6880000000003, 'maxY':..."
2,013320f6c6aa0752eb2933111106c9bcbfaaf62b416862...,여,50,일반인,분노,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 2383.9530546875003, 'maxY':...","{'boxes': {'maxX': 2382.5422949120093, 'maxY':...","{'boxes': {'maxX': 2378.7810263028277, 'maxY':..."
3,03nf73f4eb25c42388b21c54304c809be3fed9c8815eb2...,남,30,전문인,분노,공공시설/종교/의료시설,"{'boxes': {'maxX': 2065.9468000000006, 'maxY':...","{'boxes': {'maxX': 2057.5193786270265, 'maxY':...","{'boxes': {'maxX': 2044.2389975229373, 'maxY':..."
4,05s25ed5288ff5e3f2172d3c4c21b368113be833decb26...,여,20,일반인,분노,도심 환경,"{'boxes': {'maxX': 2982.1381999999994, 'maxY':...","{'boxes': {'maxX': 3035.3788594813122, 'maxY':...","{'boxes': {'maxX': 3029.6966827517267, 'maxY':..."
...,...,...,...,...,...,...,...,...,...
1495,zx1z7a73d6976eee78347847db5180b094fe70e7aa0920...,여,20,전문인,분노,상업시설/점포/시장,"{'boxes': {'maxX': 1939.2924, 'maxY': 1493.827...","{'boxes': {'maxX': 1939.2923999999998, 'maxY':...","{'boxes': {'maxX': 1916.2299061397478, 'maxY':..."
1496,zy326c174c014be2ded498c447f94dc091ff4fd0c142dd...,여,20,전문인,분노,도심 환경,"{'boxes': {'maxX': 1874.2059000000002, 'maxY':...","{'boxes': {'maxX': 1827.0454028550246, 'maxY':...","{'boxes': {'maxX': 1821.95895589011, 'maxY': 1..."
1497,zybhf86ef77e020261b33724557a6f881876051e2e6789...,여,30,전문인,분노,문화재 및 유적지,"{'boxes': {'maxX': 2363.174799999999, 'maxY': ...","{'boxes': {'maxX': 2363.1747999999993, 'maxY':...","{'boxes': {'maxX': 2363.174799999999, 'maxY': ..."
1498,zyct94aba1ee0d1e6292f6f56ecdba6d29a5b6e1ca43b6...,남,20,전문인,분노,오락/공연시설,"{'boxes': {'maxX': 2672.7248362320383, 'maxY':...","{'boxes': {'maxX': 2691.9801024658796, 'maxY':...","{'boxes': {'maxX': 2661.0699312862025, 'maxY':..."


In [17]:
happy_df = happy_df.sort_values(by='filename')
happy_df.reset_index(drop=True, inplace=True)

row_to_duplicate = happy_df.iloc[326]
happy_df = pd.concat([happy_df, row_to_duplicate.to_frame().T], ignore_index=True)
happy_df = happy_df.sort_values(by='filename')
happy_df.reset_index(drop=True, inplace=True)
happy_df

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
0,017p7f75ed075e0d375c15f5033600b9d6963fc030ff9b...,여,30,전문인,기쁨,공공시설/종교/의료시설,"{'boxes': {'maxX': 1760.6783407140142, 'maxY':...","{'boxes': {'maxX': 1746.5978670658799, 'maxY':...","{'boxes': {'maxX': 1760.7354153002582, 'maxY':..."
1,03n5c8c90453c2cc26a21a4240cc9dd20c10c1d4a01693...,남,20,일반인,기쁨,도심 환경,"{'boxes': {'maxX': 2712.7354955778005, 'maxY':...","{'boxes': {'maxX': 2761.727111823379, 'maxY': ...","{'boxes': {'maxX': 2769.799, 'maxY': 1355.2781..."
2,03pl55af0b5fb56ef753c27195e10895845517eb45556b...,여,20,전문인,기쁨,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 929.9634399999999, 'maxY': ...","{'boxes': {'maxX': 918.9989531421047, 'maxY': ...","{'boxes': {'maxX': 921.1815903948757, 'maxY': ..."
3,03xdedd73e6c6f3c1e4cca28bba7272adbb3d563327d1c...,남,30,전문인,기쁨,상업시설/점포/시장,"{'boxes': {'maxX': 2398.879, 'maxY': 1602.0869...","{'boxes': {'maxX': 2377.2676860086453, 'maxY':...","{'boxes': {'maxX': 2371.100860340365, 'maxY': ..."
4,042y0228122de0207be95453705c2239bcf202c81597bd...,여,20,일반인,기쁨,숙박 및 거주공간,"{'boxes': {'maxX': 531.72601496, 'maxY': 807.4...","{'boxes': {'maxX': 525.8980149600001, 'maxY': ...","{'boxes': {'maxX': 532.9220149600001, 'maxY': ..."
...,...,...,...,...,...,...,...,...,...
1490,zv0r61cce4a8ffdcd5158c98a1ed5624fb2ad6f92f2f76...,여,30,전문인,기쁨,상업시설/점포/시장,"{'boxes': {'maxX': 2185.566113886461, 'maxY': ...","{'boxes': {'maxX': 2174.569588899058, 'maxY': ...","{'boxes': {'maxX': 2136.0845, 'maxY': 1330.787..."
1491,zvzv1be662cdbbbafd85268fbac59430a1e2328cf62e1d...,남,40,전문인,기쁨,숙박 및 거주공간,"{'boxes': {'maxX': 2598.878855979962, 'maxY': ...","{'boxes': {'maxX': 2613.311859241855, 'maxY': ...","{'boxes': {'maxX': 2595.2258, 'maxY': 1773.428..."
1492,zx4611779ff117f718e1719768b1d627a404c02291df24...,여,30,일반인,기쁨,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 1948.291649948498, 'maxY': ...","{'boxes': {'maxX': 1966.8858603250899, 'maxY':...","{'boxes': {'maxX': 2021.0249, 'maxY': 2024.474..."
1493,zx4nc5734ce1cdb70f016d2e5a5d6b0705288105961b23...,여,50,일반인,기쁨,상업시설/점포/시장,"{'boxes': {'maxX': 1484.2194765, 'maxY': 1275....","{'boxes': {'maxX': 1477.1834764999999, 'maxY':...","{'boxes': {'maxX': 1477.5704764999998, 'maxY':..."


In [20]:
panic_df = panic_df.sort_values(by='filename')
panic_df.reset_index(drop=True, inplace=True)
row_to_duplicate = panic_df.iloc[219]
panic_df = pd.concat([panic_df, row_to_duplicate.to_frame().T], ignore_index=True)
panic_df = panic_df.sort_values(by='filename')
panic_df.reset_index(drop=True, inplace=True)
panic_df

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
0,007b10b45d424e7876e73b6b66d62d603c6ef2ee886761...,여,20,일반인,당황,숙박 및 거주공간,"{'boxes': {'maxX': 1888.9304000000002, 'maxY':...","{'boxes': {'maxX': 1888.8512363165685, 'maxY':...","{'boxes': {'maxX': 1888.9304, 'maxY': 1413.411..."
1,01hq73db2125c694f5f555f744b2020c112251215b9574...,여,40,전문인,당황,실외 자연환경,"{'boxes': {'maxX': 1968.7671, 'maxY': 1712.661...","{'boxes': {'maxX': 1968.7671, 'maxY': 1712.661...","{'boxes': {'maxX': 1968.7671000000003, 'maxY':..."
2,059hc8436a3e46efdc3c20e94dca8704846b408dfcfc39...,남,40,일반인,당황,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 2482.1235, 'maxY': 1735.228...","{'boxes': {'maxX': 2482.1234999999997, 'maxY':...","{'boxes': {'maxX': 2482.1234999999997, 'maxY':..."
3,07rq89a5d735a958d1783db31a7cd098f60969900662e1...,남,30,전문인,당황,실외 자연환경,"{'boxes': {'maxX': 993.41583, 'maxY': 1265.403...","{'boxes': {'maxX': 986.7979760834054, 'maxY': ...","{'boxes': {'maxX': 993.4158299999999, 'maxY': ..."
4,07tff4b4c1effe4a4c40c60a73eeaf1ff004ca1b48b776...,여,20,전문인,당황,도심 환경,"{'boxes': {'maxX': 1883.1353, 'maxY': 1453.875...","{'boxes': {'maxX': 1832.902042159472, 'maxY': ...","{'boxes': {'maxX': 1880.2060070707073, 'maxY':..."
...,...,...,...,...,...,...,...,...,...
1496,zyel258c8803314b24bd818b793302bfc3c3bb315a10f0...,여,20,일반인,당황,숙박 및 거주공간,"{'boxes': {'maxX': 929.3973985727096, 'maxY': ...","{'boxes': {'maxX': 936.180669291718, 'maxY': 8...","{'boxes': {'maxX': 925.4245932863153, 'maxY': ..."
1497,zyi00a1385a0ef91d4ff83ed0c29dfebfbeac00fd39f14...,남,20,일반인,당황,숙박 및 거주공간,"{'boxes': {'maxX': 2088.049209956572, 'maxY': ...","{'boxes': {'maxX': 2109.690789473684, 'maxY': ...","{'boxes': {'maxX': 2158.3557, 'maxY': 1958.946..."
1498,zz3e5aaf181c6466fe490088cce981f11044fac68f9b9f...,남,20,전문인,당황,문화재 및 유적지,"{'boxes': {'maxX': 1882.7711, 'maxY': 1693.710...","{'boxes': {'maxX': 1858.450689787159, 'maxY': ...","{'boxes': {'maxX': 1882.7710999999997, 'maxY':..."
1499,zzqj83891357fca33fbf2bcc337885ba59f32c761a06a6...,남,20,일반인,당황,도심 환경,"{'boxes': {'maxX': 1895.2123, 'maxY': 1776.566...","{'boxes': {'maxX': 1916.9879517541722, 'maxY':...","{'boxes': {'maxX': 1895.2122999999997, 'maxY':..."


In [21]:
sadness_df = sadness_df.sort_values(by='filename')
sadness_df.reset_index(drop=True, inplace=True)
sadness_df

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
0,00skd6f03c1bc8e3f618fb80b7863679bf684363a4d013...,여,20,전문인,슬픔,문화재 및 유적지,"{'boxes': {'maxX': 1624.9164, 'maxY': 1418.914...","{'boxes': {'maxX': 1621.3714458715597, 'maxY':...","{'boxes': {'maxX': 1624.9163999999998, 'maxY':..."
1,00yucd996df0c12e2dcac91e0a41d77a35181e5ff4cce6...,남,40,전문인,슬픔,실외 자연환경,"{'boxes': {'maxX': 2250.07, 'maxY': 1605.0603,...","{'boxes': {'maxX': 2250.2525267323604, 'maxY':...","{'boxes': {'maxX': 2250.1939980518337, 'maxY':..."
2,01im33663229257659a232aa1a0a15a4c1174018188107...,여,20,일반인,슬픔,도심 환경,"{'boxes': {'maxX': 2128.8220810586345, 'maxY':...","{'boxes': {'maxX': 2137.701516967979, 'maxY': ...","{'boxes': {'maxX': 2100.0532, 'maxY': 1445.638..."
3,028v4f9c9714b05ddb190ca7883cbd3e83b87fc4e7b341...,여,30,일반인,슬픔,공공시설/종교/의료시설,"{'boxes': {'maxX': 2615.803443992829, 'maxY': ...","{'boxes': {'maxX': 2604.5665895308052, 'maxY':...","{'boxes': {'maxX': 2611.3967661809243, 'maxY':..."
4,037n0dd389868164f7cf2736331868484540b863fc3fd4...,여,40,전문인,슬픔,오락/공연시설,"{'boxes': {'maxX': 1493.489, 'maxY': 1718.0852...","{'boxes': {'maxX': 1506.375099406367, 'maxY': ...","{'boxes': {'maxX': 1493.590355026364, 'maxY': ..."
...,...,...,...,...,...,...,...,...,...
1495,zuuk0561a780676e40cc61ecfd9ad664522c1668eb59a2...,남,40,전문인,슬픔,행사/사무공간,"{'boxes': {'maxX': 2607.835570165363, 'maxY': ...","{'boxes': {'maxX': 2585.59375, 'maxY': 2163, '...","{'boxes': {'maxX': 2588.385644313755, 'maxY': ..."
1496,zwujfaaef6d0a5d5d4ad708faaba727baefa8b7b521b2d...,여,30,일반인,슬픔,오락/공연시설,"{'boxes': {'maxX': 1495.5570999999998, 'maxY':...","{'boxes': {'maxX': 1508.8663791610436, 'maxY':...","{'boxes': {'maxX': 1495.5571, 'maxY': 1918.664..."
1497,zxum204e0778e08c767771d2e4dcd822a1e64dd416d5b8...,여,20,전문인,슬픔,도심 환경,"{'boxes': {'maxX': 2748.4600000000014, 'maxY':...","{'boxes': {'maxX': 2715.928435361856, 'maxY': ...","{'boxes': {'maxX': 2723.0488490981134, 'maxY':..."
1498,zygcc2ae65d241fa88e458f5a933aa46a4e3f1d666d654...,남,20,전문인,슬픔,상업시설/점포/시장,"{'boxes': {'maxX': 1032.8927, 'maxY': 1335.293...","{'boxes': {'maxX': 1023.3577442403264, 'maxY':...","{'boxes': {'maxX': 1017.5351580627035, 'maxY':..."


In [12]:
# 디렉토리 내 이미지 파일 경로
directory_path = '/content/final_data/img/train/anger'
image_files = sorted(os.listdir(directory_path))

def save_region_as_image(original_image, save_directory, file_name):
    try:
        image = original_image
        # 좌표 가져오기
        maxX = anger_df['annot_A'][i]['boxes']['maxX']
        maxY = anger_df['annot_A'][i]['boxes']['maxY']
        minX = anger_df['annot_A'][i]['boxes']['minX']
        minY = anger_df['annot_A'][i]['boxes']['minY']

        cropped_image = original_image[int(minY):int(maxY), int(minX):int(maxX)]
        resized_image = cv2.resize(cropped_image, (256, 256))

        # 이미지 저장 경로 설정
        save_path = os.path.join(save_directory, file_name)

        # 이미지 저장
        cv2.imwrite(save_path, resized_image)
    except Exception as e:
        print(f"Error occurred while processing {file_name}: {e}")


for i, file_name in enumerate(image_files):
    if file_name in image_files:
        original_image_path = os.path.join(directory_path, file_name)
        original_image = cv2.imread(original_image_path)
        save_directory = '/content/anger_crop/'
        save_file_name = f'anger{i+1}.jpg'
        save_region_as_image(original_image, save_directory, save_file_name)
    else:
        print(f"Image file for {file_name} not found.")

Error occurred while processing anger405.jpg: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error occurred while processing anger495.jpg: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [18]:
# 디렉토리 내 이미지 파일 경로
directory_path = '/content/final_data/img/train/happy'
image_files = sorted(os.listdir(directory_path))

def save_region_as_image(original_image, save_directory, file_name):
    try:
        image = original_image
        # 좌표 가져오기
        maxX = happy_df['annot_A'][i]['boxes']['maxX']
        maxY = happy_df['annot_A'][i]['boxes']['maxY']
        minX = happy_df['annot_A'][i]['boxes']['minX']
        minY = happy_df['annot_A'][i]['boxes']['minY']

        cropped_image = original_image[int(minY):int(maxY), int(minX):int(maxX)]
        resized_image = cv2.resize(cropped_image, (256, 256))

        # 이미지 저장 경로 설정
        save_path = os.path.join(save_directory, file_name)

        # 이미지 저장
        cv2.imwrite(save_path, resized_image)
    except Exception as e:
        print(f"Error occurred while processing {file_name}: {e}")


for i, file_name in enumerate(image_files):
    if file_name in image_files:
        original_image_path = os.path.join(directory_path, file_name)
        original_image = cv2.imread(original_image_path)
        save_directory = '/content/happy_crop/'
        save_file_name = f'happy{i+1}.jpg'
        save_region_as_image(original_image, save_directory, save_file_name)
    else:
        print(f"Image file for {file_name} not found.")

Error occurred while processing happy138.jpg: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [23]:
# 디렉토리 내 이미지 파일 경로
directory_path = '/content/final_data/img/train/panic'
image_files = sorted(os.listdir(directory_path))

def save_region_as_image(original_image, save_directory, file_name):
    try:
        image = original_image
        # 좌표 가져오기
        maxX = panic_df['annot_A'][i]['boxes']['maxX']
        maxY = panic_df['annot_A'][i]['boxes']['maxY']
        minX = panic_df['annot_A'][i]['boxes']['minX']
        minY = panic_df['annot_A'][i]['boxes']['minY']

        cropped_image = original_image[int(minY):int(maxY), int(minX):int(maxX)]
        resized_image = cv2.resize(cropped_image, (256, 256))

        # 이미지 저장 경로 설정
        save_path = os.path.join(save_directory, file_name)

        # 이미지 저장
        cv2.imwrite(save_path, resized_image)
    except Exception as e:
        print(f"Error occurred while processing {file_name}: {e}")


for i, file_name in enumerate(image_files):
    if file_name in image_files:
        original_image_path = os.path.join(directory_path, file_name)
        original_image = cv2.imread(original_image_path)
        save_directory = '/content/panic_crop/'
        save_file_name = f'panic{i+1}.jpg'
        save_region_as_image(original_image, save_directory, save_file_name)
    else:
        print(f"Image file for {file_name} not found.")

Error occurred while processing panic152.jpg: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error occurred while processing panic1410.jpg: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [ ]:
# 디렉토리 내 이미지 파일 경로
directory_path = '/content/final_data/img/train/sadness'
image_files = sorted(os.listdir(directory_path))

def save_region_as_image(original_image, save_directory, file_name):
    try:
        image = original_image
        # 좌표 가져오기
        maxX = sadness_df['annot_A'][i]['boxes']['maxX']
        maxY = sadness_df['annot_A'][i]['boxes']['maxY']
        minX = sadness_df['annot_A'][i]['boxes']['minX']
        minY = sadness_df['annot_A'][i]['boxes']['minY']

        cropped_image = original_image[int(minY):int(maxY), int(minX):int(maxX)]
        resized_image = cv2.resize(cropped_image, (256, 256))

        # 이미지 저장 경로 설정
        save_path = os.path.join(save_directory, file_name)

        # 이미지 저장
        cv2.imwrite(save_path, resized_image)
    except Exception as e:
        print(f"Error occurred while processing {file_name}: {e}")


for i, file_name in enumerate(image_files):
    if file_name in image_files:
        original_image_path = os.path.join(directory_path, file_name)
        original_image = cv2.imread(original_image_path)
        save_directory = '/content/sadness_crop/'
        save_file_name = f'sadness{i+1}.jpg'
        save_region_as_image(original_image, save_directory, save_file_name)
    else:
        print(f"Image file for {file_name} not found.")

Error occurred while processing sadness36.jpg: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [24]:
import shutil

# 압축할 디렉토리 경로
directory_to_zip = '/content/sadness_crop'

# 저장할 zip 파일 경로와 이름
zip_file_path = '/content/sadness_crop.zip'

# 디렉토리를 zip 파일로 압축
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', directory_to_zip)

print(f'{zip_file_path}로 압축 완료.')

/content/panic_crop.zip로 압축 완료.
